In [1]:
from FileReader_impl import FileReader
from TextChunker_impl import TextChunker
from TextEncoder_impl import TextEmbedding
from TextRankBm25_impl import TextRankBm25
from TextCrossEncoder_impl import TextCrossEncoder

from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

from pymilvus import connections, db, utility, FieldSchema, DataType, Collection, CollectionSchema
import json
import psycopg2


from openai import OpenAI

2024-11-25 16:53:48.519110: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1732542828.640971    1704 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1732542828.674657    1704 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-25 16:53:48.995463: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
query = 'Кто был президентом США в середине 20-го века?'
document = FileReader('1.txt').load_document()
chunks = TextChunker().split_text(document)
Data_db = TextEmbedding().vectorize_text(chunks)

/home/pavel/miniconda3/envs/retriver_search/lib/python3.11/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11060). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0


In [3]:
def check_db(array, name_db):
    check = 0
    for arr in array:
        if arr == name_db:
            check += 1
    return check

In [4]:
def check_coll(array, name_coll):
    for arr in array:
        if arr == name_coll:
            collection = Collection(name=name_coll)
            collection.drop()

In [5]:
baza = 'TEST_2'
size_vec = 1024
col_data = 'txt_1'

conn = connections.connect(host='localhost', port=19530) # Подключение к базе данных Milvus
arr = db.list_database() # Список созданных баз данных в Milvis
if not check_db(arr, baza): # Проверка на существование базы данных
    db.create_database(baza) # Создание базы данных
db.using_database(baza) # Указание на использование базы данных

In [6]:
arr = utility.list_collections() # Список коллекций
check_coll(arr, col_data) # Проверка на существование коллекции

In [7]:
## Создание колекции и схемы этой колекции
id_field = FieldSchema(name='id', dtype=DataType.INT64, is_primary=True, auto_id=False)
source_field = FieldSchema(name='source', dtype=DataType.VARCHAR, max_length=255, is_primary=False)
embedding_field = FieldSchema(name='embeddings', dtype=DataType.FLOAT_VECTOR, dim=size_vec)
content_field = FieldSchema(name='content', dtype=DataType.VARCHAR, max_length=2048)

schema = CollectionSchema(fields=[id_field, source_field, embedding_field, content_field])
collection = Collection(name=col_data, schema=schema)

In [8]:
## Создание индекса поиска
index_params = {
    "metric_type": "COSINE",
    'index_type': 'IVF_FLAT',
    "params":{"nlist": 5}
}

collection.create_index(field_name='embeddings', index_params=index_params)
collection.load()

collection.insert([Data_db['id'], Data_db['source'], Data_db['emb'], Data_db['content']]) # Загружаем наши данные в базу данных

(insert count: 33, delete count: 0, upsert count: 0, timestamp: 454175719546945582, success count: 33, err count: 0

In [9]:
def search_milvus(query, collection, limit=10):
    model = TextEmbedding().model_emb()
    query_embedding = model.encode(query)
    search_params = index_params
    results = collection.search(
        data = [query_embedding],
        anns_field = 'embeddings',
        param = search_params,
        limit = limit,
        output_fields = ['source', 'content']
    )
    return results

results = search_milvus(query, collection)
i=1
for result in results[0]:
        print(f"{i}: {result.entity}")
        i+=1

1: id: 21, distance: 0.8355264067649841, entity: {'source': '1.txt', 'content': 'подтекст. 5 марта 1946 г. бывший премьер-министр Великобритании Черчилль, приехавший в США, в присутствии президента США Г. Трумэна произнёс свою известную речь. Он говорил о разделении Европы'}
2: id: 16, distance: 0.8339384198188782, entity: {'source': '1.txt', 'content': 'из-за войны Великобритания и Франция не имели сил для решения таких задач. Роль лидера западного мира перешла к США, которые в ходе войны превратились в самую значительную экономическую силу,'}
3: id: 17, distance: 0.8225345611572266, entity: {'source': '1.txt', 'content': 'в самую значительную экономическую силу, обладавшую к тому же ядерным оружием. В 1947 г. президент США Г. Трумэн добился от конгресса согласия на оказание значительной помощи Греции (правительство'}
4: id: 22, distance: 0.8089385032653809, entity: {'source': '1.txt', 'content': 'известную речь. Он говорил о разделении Европы «железным занавесом» и призвал противодей

In [10]:
def filter_results(results):
    data = {}
    ids = []
    distances = []
    sources = []
    contents = []

    for result in results[0]:
        id = result.id
        distance = result.distance
        source = result.entity.get('source')
        content = result.entity.get('content')

        ids.append(id)
        distances.append(distance)
        sources.append(source)
        contents.append(content)
    
    data['id'] = ids
    data['distance'] = distances
    data['source'] = sources
    data['content'] = contents

    return data

Data_search = filter_results(results)
Data_search

{'id': [21, 16, 17, 22, 12, 15, 20, 19, 2, 24],
 'distance': [0.8355264067649841,
  0.8339384198188782,
  0.8225345611572266,
  0.8089385032653809,
  0.8076500296592712,
  0.8068890571594238,
  0.8054627180099487,
  0.8051764965057373,
  0.8012842535972595,
  0.8005954027175903],
 'source': ['1.txt',
  '1.txt',
  '1.txt',
  '1.txt',
  '1.txt',
  '1.txt',
  '1.txt',
  '1.txt',
  '1.txt',
  '1.txt'],
 'content': ['подтекст. 5 марта 1946 г. бывший премьер-министр Великобритании Черчилль, приехавший в США, в присутствии президента США Г. Трумэна произнёс свою известную речь. Он говорил о разделении Европы',
  'из-за войны Великобритания и Франция не имели сил для решения таких задач. Роль лидера западного мира перешла к США, которые в ходе войны превратились в самую значительную экономическую силу,',
  'в самую значительную экономическую силу, обладавшую к тому же ядерным оружием. В 1947 г. президент США Г. Трумэн добился от конгресса согласия на оказание значительной помощи Греции (правит

In [11]:
conn = psycopg2.connect(host = 'localhost', dbname = 'postgres', user = 'postgres', password='qwerty', port=5432)
cur = conn.cursor()

cur.execute("""
    CREATE TABLE IF NOT EXISTS file_data (
        id INT PRIMARY KEY,
        source VARCHAR(255),
        content VARCHAR(2048)            
    );
""")

Data_postgres = [
    (i + 1, chunk.metadata['source'], chunk.page_content)
    for i, chunk in enumerate(chunks)
]

# cur.executemany("INSERT INTO file_data (id, source, content) VALUES (%s, %s, %s)", Data_postgres)
# cur.execute("DROP TABLE file_data;")
conn.commit()

cur.execute("SELECT id FROM file_data")
ids = cur.fetchall()

cur.execute("SELECT source FROM file_data")
sources = cur.fetchall()

cur.execute("SELECT content FROM file_data")
contents = cur.fetchall()

data = {
    'id': [id[0] for id in ids],
    'source': [source[0] for source in sources],
    'content': [content[0] for content in contents]
}

bm25 = TextRankBm25(data)
scores = bm25.bm25_search(query)
results = bm25.choice_not_zero(scores)
bm25.bubble_sorting(results)
bm25.res_output(results)

cur.close()
conn.close()

19: 4.480075217915705
эту инициативу, означала, что в послевоенном мире США были готовы активно противодействовать советскому влиянию при помощи утверждения собственной гегемонии. Для развития этого курса был разработан
------------------------------------------------

20: 3.7145827020418345
Для развития этого курса был разработан «план Маршалла» — программа масштабной финансовой помощи США Европе, которая имела очевидный политический подтекст. 5 марта 1946 г. бывший премьер-министр
------------------------------------------------

21: 2.797561048411508
подтекст. 5 марта 1946 г. бывший премьер-министр Великобритании Черчилль, приехавший в США, в присутствии президента США Г. Трумэна произнёс свою известную речь. Он говорил о разделении Европы
------------------------------------------------

28: 2.797561048411508
тяжёлыми были потери в людях. Война оставила миллионы инвалидов, сирот, вдов, принесла горе почти в каждую семью. Гигантский урон понесла топливно-энергетическая база страны. 

In [12]:
data_milv = Data_search['id']
data_postgres = results['id']
print(data_milv)
print(data_postgres)

[21, 16, 17, 22, 12, 15, 20, 19, 2, 24]
[19, 20, 21, 28, 29, 17, 30, 5, 6, 9, 15, 11, 16, 3, 14, 31, 12, 4, 27, 18, 24, 8, 1]


In [13]:
def checking_match(data_milv, data_postgres):
    data = []  
    for id in data_milv:
        data.append(id)
    for id_p in data_postgres:
        i=0
        for id_m in data_milv:
            if id_p != id_m:
                i += 1
        if i == len(data_milv):
            data.append(id_p)
    return data
data_id = checking_match(data_milv, data_postgres)
print(data_id)

[21, 16, 17, 22, 12, 15, 20, 19, 2, 24, 28, 29, 30, 5, 6, 9, 11, 3, 14, 31, 4, 27, 18, 8, 1]


In [14]:
cross_encoder = TextCrossEncoder().calculate_relevance(query, Data_db, data_id)
print(cross_encoder)

[21, 17, 22, 4, 8, 20, 15, 1, 6, 12, 5, 31, 11, 24, 3, 2, 19, 18, 28, 29, 9, 27, 14, 16, 30]


In [20]:
def generate_prompt(query, result_id):
    instructions = """
        Ваша роль - выступать в качестве системы информационного поиска. 
    Вам будет задан вопрос, а также предоставлены релевантные отрывки из различных документов. 
    Ваша задача - сформировать короткий и информативный ответ (не более 150 слов), основанный исключительно на представленных отрывках. 
    Обязательно использовать информацию только из данных отрывков.
    Важно соблюдать нейтральный и объективный тон, а также избегать повторения текста.
    В конце формируйте окончательный ответ.
    Не пытайтесь изобрести ответ.
    Отвечайте исключительно на русском языке, за исключением специфических терминов.
    Если представленные документы не содержат информации, достаточной для формирования ответа, скажите: "Я не могу ответить на Ваш вопрос, используя информацию из предоставленной документации.
    Попробуйте переформулировать вопрос."
    Если документ содержит информацию, относящуюся к запросу, но запрос не предполагает прямого ответа, то просто перескажите содержание релевантного документа. 
    Вопрос:"""

    promt = [instructions, query]
    
    i=0
    for id in result_id:
        promt.append(f"\nФрагмент {i+1}: {Data_db['content'][id-1]}")
        i += 1

    return ' '.join(promt)

In [21]:
promt = generate_prompt(query, cross_encoder)
print(promt)


        Ваша роль - выступать в качестве системы информационного поиска. 
    Вам будет задан вопрос, а также предоставлены релевантные отрывки из различных документов. 
    Ваша задача - сформировать короткий и информативный ответ (не более 150 слов), основанный исключительно на представленных отрывках. 
    Обязательно использовать информацию только из данных отрывков.
    Важно соблюдать нейтральный и объективный тон, а также избегать повторения текста.
    В конце формируйте окончательный ответ.
    Не пытайтесь изобрести ответ.
    Отвечайте исключительно на русском языке, за исключением специфических терминов.
    Если представленные документы не содержат информации, достаточной для формирования ответа, скажите: "Я не могу ответить на Ваш вопрос, используя информацию из предоставленной документации.
    Попробуйте переформулировать вопрос."
    Если документ содержит информацию, относящуюся к запросу, но запрос не предполагает прямого ответа, то просто перескажите содержание рел